In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter

# -----------------------------------------------------------------------
# 1. Load Data
# -----------------------------------------------------------------------
data = pd.read_csv('./TABLE1.csv')  # ใส่ชื่อไฟล์จริง
# คาดว่ามีคอลัมน์: Eid, Agency_Name, Subject_Field, Publish_month, Year, Country, Title, Index_Terms

# -----------------------------------------------------------------------
# 2. Data Cleaning / Handling duplicates if needed
# -----------------------------------------------------------------------
# ลบ duplicates หากมี
data = data.drop_duplicates()

# ตรวจสอบ Missing Values ใน Index_Terms
data['Index_Terms'] = data['Index_Terms'].fillna("").astype(str)

# -----------------------------------------------------------------------
# 3. Split data by Eid to avoid data leakage
# -----------------------------------------------------------------------
unique_eids = data['Eid'].unique()
train_eids, test_eids = train_test_split(unique_eids, test_size=0.2, random_state=42)

train_df = data[data['Eid'].isin(train_eids)].reset_index(drop=True)
test_df = data[data['Eid'].isin(test_eids)].reset_index(drop=True)

# -----------------------------------------------------------------------
# 4. Encode Target (Agency_Name)
# -----------------------------------------------------------------------
le_agency = LabelEncoder()
train_df['Agency_label'] = le_agency.fit_transform(train_df['Agency_Name'])

# สำหรับ test set ถ้ามี Agency_Name ที่ไม่เคยเจอใน train ให้ลบออก
test_df = test_df[test_df['Agency_Name'].isin(le_agency.classes_)]
test_df['Agency_label'] = le_agency.transform(test_df['Agency_Name'])

# -----------------------------------------------------------------------
# 5. Prepare Features
# รวม Title กับ Index_Terms เข้าด้วยกันเป็นฟีเจอร์ข้อความเดียว
train_df['text_all'] = train_df['Title'].fillna("") + " " + train_df['Index_Terms'].fillna("")
test_df['text_all'] = test_df['Title'].fillna("") + " " + test_df['Index_Terms'].fillna("")

X_train = train_df[['text_all', 'Subject_Field', 'Publish_month', 'Year', 'Country']]
y_train = train_df['Agency_label']

X_test = test_df[['text_all', 'Subject_Field', 'Publish_month', 'Year', 'Country']]
y_test = test_df['Agency_label']

# -----------------------------------------------------------------------
# 6. Column Specification
# text_all จะใช้ TF-IDF
# Subject_Field, Country เป็นหมวดหมู่ (Categorical)
# Publish_month, Year เป็นตัวเลข (Numeric)
text_feature = 'text_all'
cat_features = ['Subject_Field', 'Country']
num_features = ['Publish_month', 'Year']

# -----------------------------------------------------------------------
# 7. Create Preprocessing Pipeline
# -----------------------------------------------------------------------
text_transformer = TfidfVectorizer(stop_words='english', max_features=1000)
cat_transformer = OneHotEncoder(handle_unknown='ignore')
num_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, text_feature),
        ('cat', cat_transformer, cat_features),
        ('num', num_transformer, num_features)
    ],
    remainder='drop'
)

# -----------------------------------------------------------------------
# 8. Handle Class Imbalance (If needed)
# -----------------------------------------------------------------------
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = {c: w for c, w in zip(classes, class_weights)}

# -----------------------------------------------------------------------
# 9. Build Pipeline with Model
# -----------------------------------------------------------------------
model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('clf', RandomForestClassifier(n_estimators=200, random_state=42, class_weight=class_weight_dict))
])

# -----------------------------------------------------------------------
# 10. Train Model
# -----------------------------------------------------------------------
model.fit(X_train, y_train)

# -----------------------------------------------------------------------
# 11. Predict & Evaluate
# -----------------------------------------------------------------------
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le_agency.classes_))

# -----------------------------------------------------------------------
# หมายเหตุ:
# - การมี Country ซ้ำไม่ถือว่าเป็น Data Leakage เนื่องจากเป็นคุณลักษณะตามธรรมชาติของข้อมูล:
#   หลาย Agency อาจมาจากประเทศเดียวกัน เป็นข้อมูลที่มีใน train ก็อาจมีใน test ได้ ไม่ได้เปิดเผยอะไรที่ทำให้
#   โมเดลเห็นข้อมูลเฉพาะของ test ล่วงหน้า
# - การป้องกัน Data Leakage สำคัญคือต้องไม่ให้ Eid เดียวกันปรากฏในทั้ง train และ test
# - หากต้องการปรับปรุงความแม่นยำ อาจลองปรับพารามิเตอร์ TF-IDF, โมเดล หรือทำ Feature Engineering เพิ่มเติม
